# Sprint14課題 ディープラーニングフレームワーク1

## この課題の目的

- フレームワークのコードを読めるようにする
- フレームワークを習得し続けられるようになる
- 理論を知っている範囲をフレームワークで動かす

## コードリーディング

TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。

tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。

## 【問題1】スクラッチを振り返る

ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

（例）

- 重みを初期化する必要があった
- エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう

- ハイパーパラメータの設定
- 重み、バイアスを初期化
- 順伝播→誤差の算出→逆伝播→勾配降下法による重み・バイアスの更新をミニバッチごとに行い、設定したエポック分繰り返す。

### データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。

[Iris Species](https://www.kaggle.com/uciml/iris/data)

目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。

## 【問題2】スクラッチとTensorFlowの対応を考える

以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。

それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
'''
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
'''
#データセットの読み込み
dataset_path = 'Iris.csv'
df = pd.read_csv(dataset_path)

#データフレームから条件抽出
df = df[(df['Species'] == 'Iris-versicolor') | (df['Species'] == 'Iris-virginica')]
y = df['Species']
X = df.loc[:, ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
y = np.array(y)
X = np.array(X)

#ラベルを数値に変換
y[y == 'Iris-versicolor'] = 0
y[y == 'Iris-virginica'] = 1

#次元変換
y = y.astype(np.int)[:, np.newaxis]

#trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [3]:
class GetMiniBatch:
    '''
    ミニバッチを取得するイテレータ
    
    Parameters
    --------------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    '''
    def __init__(self, X, y, batch_size=10, seed=0):
        self.batch_size = batch_size
        
        #配列をシャッフル
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        
        self._stop = np.ceil(X.shape[0] / self.batch_size).astype(np.int)
        
    
    def __len__(self):
        #len
        return self._stop
    
    
    def __getitem__(self, item):
        #要素の参照
        p0 = item * self.batch_size
        p1 = item * self.batch_size + self.batch_size
        return self.X[p0: p1], self.y[p0: p1]
    
    
    def __iter__(self):
        #イテレートする
        self._counter = 0
        return self
    
    
    def __next__(self):
        #__iter__から次の値を取得
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter * self.batch_size
        p1 = self._counter * self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0: p1], self.y[p0: p1]

In [4]:
#ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 100
n_hidden2 = 50
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

#計算グラフに渡す引数の形を決める
X = tf.placeholder('float', [None, n_input])
Y = tf.placeholder('float', [None, n_classes])

#trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [5]:
def example_net(x):
    '''
    単純な3層ニューラルネットワーク
    '''
    #重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    #全結合層
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] #addと同じ
    
    return layer_output

In [6]:
#ネットワーク構造の読み込み
logits = example_net(X)

#目的関数(クロスエントロピー)
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

#最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

#指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#variableの初期値
init = tf.global_variables_initializer()

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
#計算グラフの実行
with tf.Session() as sess:
    
    #変数を初期化
    sess.run(init)
    
    #エポックごとにループ
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0] / batch_size).astype(np.int)
        
        #lossとaccの初期値
        total_loss = 0
        total_acc = 0
        
        #ミニバッチごとにループ
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # 誤差を最小にする
            sess.run(train_op, feed_dict={X: X_train, Y: y_train})
            
            #lossとaccuracyを求める
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        
        #サンプル数で割る
        total_loss /= n_samples
        total_acc /= n_samples
        
        #検証用データの推定
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print('Epoch{}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}'.format(epoch + 1, loss, val_loss, acc, val_acc))
        
    #未知データの推定    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print('test_acc : {:.3f}'.format(test_acc))

Epoch1, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
Epoch2, loss : 0.0000, val_loss : 0.6402, acc : 1.000, val_acc : 0.938
Epoch3, loss : 0.0000, val_loss : 0.0045, acc : 1.000, val_acc : 1.000
Epoch4, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
Epoch5, loss : 0.0000, val_loss : 0.1114, acc : 1.000, val_acc : 0.938
Epoch6, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
Epoch7, loss : 0.0000, val_loss : 0.0215, acc : 1.000, val_acc : 1.000
Epoch8, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
Epoch9, loss : 0.0000, val_loss : 0.0045, acc : 1.000, val_acc : 1.000
Epoch10, loss : 0.0000, val_loss : 0.0209, acc : 1.000, val_acc : 1.000
test_acc : 0.900


- 順伝播→誤差の算出→逆伝播→勾配降下法による重み・バイアスの更新をミニバッチごとに行い、設定したエポック分繰り返す。
    
    - プレースホルダー  
    tf.placeholder()で型を作って、そこにデータを供給。
    
    - 重み、バイアスの用意  
    tf.variable()で変数を定義
    
    - 目的関数、勾配降下法を定義
    
    - 重み、バイアスの初期化  
    tf.global_variables_initializer()で変数を初期化
    
    - 実行  
    tf.Session()をインスタンス化し、run()で実行、もしくはWith構文を使う(With tf.Session() as sess: の形)  
    run()の中でfeed_dict={x : x_train}とするとプレースホルダーxにx_trainの値が入る。


## 他のデータセットへの適用
これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。

- Iris（3種類全ての目的変数を使用）
- House Prices

どのデータセットもtrain, val, testの3種類に分けて使用してください。

## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
[Iris Species](https://www.kaggle.com/uciml/iris/data)

サンプルコードと同じくこの中のtrain.csvを使用してください。目的変数はSpeciesに含まれる3種類全てを使います。

2クラスの分類と3クラス以上の分類の違いを考慮してください。

それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

ヒント

以下の2箇所は2クラス分類特有の処理です。

メソッドは以下のように公式ドキュメントを確認してください。

[tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits)

[tf.math.sign  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/math/sign)

＊tf.signとtf.math.signは同じ働きをします。

In [8]:
#データセットの読み込み
dataset_path = 'Iris.csv'
df = pd.read_csv(dataset_path)

#nparrayに変換
y = df['Species']
X = df.loc[:, ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
y = np.array(y)
X = np.array(X)

#ラベルを数値に変換
y[y == 'Iris-setosa'] = 0
y[y == 'Iris-versicolor'] = 1
y[y == 'Iris-virginica'] = 2

#次元変換
y = y.astype(np.int)[:, np.newaxis]

#one-hotエンコーディング
enc = OneHotEncoder(categories='auto')
y = enc.fit_transform(y).A

#trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [9]:
#ハイパーパラメータの設定
learning_rate = 0.001 #学習率
batch_size = 10 #バッチサイズ
num_epochs = 10 #エポック数
n_hidden1 = 100  #1層目
n_hidden2 = 50 #2層目
n_input = X_train.shape[1] #特徴量数
n_samples = X_train.shape[0] #サンプル数
n_classes = 3 #クラス数

#計算グラフに渡す引数の形を決める
X = tf.placeholder('float', [None, n_input])
Y = tf.placeholder('float', [None, n_classes])

#trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [10]:
def example_net(x):
    '''
    単純な3層ニューラルネットワーク
    '''
    #重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    #全結合層
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] #addと同じ
    
    return layer_output

In [11]:
#ネットワーク構造の読み込み
logits = example_net(X)

#目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

#最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(tf.nn.softmax(logits), axis=1))

#指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#variableの初期値
init = tf.global_variables_initializer()

In [12]:
#計算グラフの実行
with tf.Session() as sess:
    
    #変数を初期化
    sess.run(init)
    
    #エポックごとにループ
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0] / batch_size).astype(np.int)
        
        #lossとaccの初期値
        total_loss = 0
        total_acc = 0
        
        #ミニバッチごとにループ
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # 誤差を最小にする
            sess.run(train_op, feed_dict={X: X_train, Y: y_train})
            
            #lossとaccuracyを求める
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        
        #サンプル数で割る
        total_loss /= n_samples
        total_acc /= n_samples
        
        #検証用データの推定
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print('Epoch{}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}'.format(epoch + 1, loss, val_loss, acc, val_acc))
        
    #未知データの推定    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print('test_acc : {:.3f}'.format(test_acc))

Epoch1, loss : 292.1176, val_loss : 310.5425, acc : 0.167, val_acc : 0.375
Epoch2, loss : 177.9974, val_loss : 222.8857, acc : 0.167, val_acc : 0.375
Epoch3, loss : 76.3792, val_loss : 155.0103, acc : 0.167, val_acc : 0.208
Epoch4, loss : 54.2091, val_loss : 119.4702, acc : 0.333, val_acc : 0.292
Epoch5, loss : 44.9720, val_loss : 79.9847, acc : 0.167, val_acc : 0.375
Epoch6, loss : 20.5114, val_loss : 43.3267, acc : 0.667, val_acc : 0.500
Epoch7, loss : 6.6078, val_loss : 10.6003, acc : 0.833, val_acc : 0.625
Epoch8, loss : 0.0000, val_loss : 0.2043, acc : 1.000, val_acc : 0.958
Epoch9, loss : 0.0002, val_loss : 0.9315, acc : 1.000, val_acc : 0.917
Epoch10, loss : 0.0000, val_loss : 0.3163, acc : 1.000, val_acc : 0.958
test_acc : 1.000


## 【問題4】House Pricesのモデルを作成

回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。

[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。

分類問題と回帰問題の違いを考慮してください。

In [13]:
#データセットの読み込み
dataset_path = 'train.csv'
df = pd.read_csv(dataset_path)

#nparrayに変換
y = df['SalePrice']
X = df.loc[:, ['GrLivArea', 'YearBuilt']]
y = np.array(y)
X = np.array(X)

#次元変換
y = y[:, np.newaxis]

#対数変換
X = np.log(X)
y = np.log(y)

#trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [14]:
#ハイパーパラメータの設定
learning_rate = 0.005 #学習率
batch_size = 20 #バッチサイズ
num_epochs = 20 #エポック数
n_hidden1 = 100 #１層目
n_hidden2 = 50 #2層目
n_input = X_train.shape[1] #特徴量数
n_samples = X_train.shape[0] #サンプル数
n_classes = 1 #クラス数

#計算グラフに渡す引数の形を決める
X = tf.placeholder('float', [None, n_input])
Y = tf.placeholder('float', [None, n_classes])

#trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [15]:
def example_net(x):
    '''
    単純な3層ニューラルネットワーク
    '''
    #重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    #全結合層
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] #addと同じ
    
    return layer_output

In [16]:
#ネットワーク構造の読み込み
logits = example_net(X)

#目的関数
loss_op = tf.reduce_mean(tf.square(Y - logits))

#最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#推定結果
pred = logits

#指標値の算出
mse = tf.reduce_mean(tf.square(Y - pred))
r2 =  1 - (tf.reduce_sum(tf.square(Y - pred)) / tf.reduce_sum(tf.square(Y - tf.reduce_mean(Y))))

#variableの初期値
init = tf.global_variables_initializer()

In [17]:
#計算グラフの実行
with tf.Session() as sess:
    
    #変数を初期化
    sess.run(init)
    
    #エポックごとにループ
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0] / batch_size).astype(np.int)
        
        #lossの初期値
        total_loss = 0

        #ミニバッチごとにループ
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # 誤差を最小にする
            sess.run(train_op, feed_dict={X: X_train, Y: y_train})
            
            #lossを求める
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        
        #サンプル数で割る
        total_loss /= n_samples
        
        #検証用データの推定
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print('Epoch{}, loss : {:.4f}, val_loss : {:.4f}'.format(epoch + 1, loss, val_loss))
        
    #未知データの推定    
    test_mse = sess.run(mse, feed_dict={X: X_test, Y: y_test})
    test_r2 = sess.run(r2, feed_dict={X: X_test, Y: y_test})

    print('test_mse : {:.3f}'.format(test_mse))
    print('test_R2 : {:.3f}'.format(test_r2))

Epoch1, loss : 85.6418, val_loss : 72.2104
Epoch2, loss : 46.5566, val_loss : 40.0764
Epoch3, loss : 27.3224, val_loss : 22.7694
Epoch4, loss : 15.6235, val_loss : 13.1274
Epoch5, loss : 5.3371, val_loss : 4.2955
Epoch6, loss : 0.5233, val_loss : 0.8122
Epoch7, loss : 0.1330, val_loss : 0.2777
Epoch8, loss : 0.0697, val_loss : 0.1669
Epoch9, loss : 0.0629, val_loss : 0.1243
Epoch10, loss : 0.0610, val_loss : 0.1014
Epoch11, loss : 0.0603, val_loss : 0.0874
Epoch12, loss : 0.0600, val_loss : 0.0767
Epoch13, loss : 0.0599, val_loss : 0.0699
Epoch14, loss : 0.0598, val_loss : 0.0664
Epoch15, loss : 0.0597, val_loss : 0.0654
Epoch16, loss : 0.0597, val_loss : 0.0652
Epoch17, loss : 0.0597, val_loss : 0.0652
Epoch18, loss : 0.0597, val_loss : 0.0652
Epoch19, loss : 0.0597, val_loss : 0.0653
Epoch20, loss : 0.0597, val_loss : 0.0652
test_mse : 0.067
test_R2 : 0.557


## 【問題5】MNISTのモデルを作成

ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。

3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。

スクラッチで実装したモデルの再現を目指してください。

In [18]:
#データの読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#次元変換
y_train = y_train.astype(np.int)[:, np.newaxis]
y_test = y_test.astype(np.int)[:, np.newaxis]

#one-hotエンコーディング
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)

#データの変換
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

#分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [19]:
#ハイパーパラメータの設定
learning_rate = 0.01 #学習率
batch_size = 100 #バッチサイズ
num_epochs = 10 #エポック数
n_hidden1 = 50 #1層目
n_hidden2 = 25 #2層目
n_input = X_train.shape[1] #特徴量数
n_samples = X_train.shape[0] #サンプル数
n_classes = 10 #クラス数

#計算グラフに渡す引数の形を決める
X = tf.placeholder('float', [None, n_input])
Y = tf.placeholder('float', [None, n_classes])

#trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [20]:
def example_net(x):
    '''
    単純な3層ニューラルネットワーク
    '''
    #重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    #全結合層
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] #addと同じ
    
    return layer_output

In [21]:
#ネットワーク構造の読み込み
logits = example_net(X)

#目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

#最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(tf.nn.softmax(logits), axis=1))

#指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#variableの初期値
init = tf.global_variables_initializer()

In [22]:
#計算グラフの実行
with tf.Session() as sess:
    
    #変数を初期化
    sess.run(init)
    
    #エポックごとにループ
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0] / batch_size).astype(np.int)
        
        #lossとaccの初期値
        total_loss = 0
        total_acc = 0
        
        #ミニバッチごとにループ
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # 誤差を最小にする
            sess.run(train_op, feed_dict={X: X_train, Y: y_train})
            
            #lossとaccuracyを求める
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        
        #バッチ数で割る
        total_loss /= total_batch
        total_acc /= total_batch
        
        #検証用データの推定
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print('Epoch{}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}'.format(epoch + 1, loss, val_loss, acc, val_acc))
        
    #未知データの推定    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print('test_acc : {:.3f}'.format(test_acc))

Epoch1, loss : 1.1428, val_loss : 1.1479, acc : 0.560, val_acc : 0.585
Epoch2, loss : 0.6695, val_loss : 0.6067, acc : 0.780, val_acc : 0.831
Epoch3, loss : 0.4560, val_loss : 0.4810, acc : 0.840, val_acc : 0.879
Epoch4, loss : 0.3518, val_loss : 0.4315, acc : 0.870, val_acc : 0.898
Epoch5, loss : 0.3104, val_loss : 0.4190, acc : 0.920, val_acc : 0.909
Epoch6, loss : 0.2397, val_loss : 0.4191, acc : 0.950, val_acc : 0.917
Epoch7, loss : 0.2185, val_loss : 0.4386, acc : 0.970, val_acc : 0.919
Epoch8, loss : 0.1994, val_loss : 0.4670, acc : 0.990, val_acc : 0.921
Epoch9, loss : 0.1993, val_loss : 0.5410, acc : 0.990, val_acc : 0.920
Epoch10, loss : 0.1710, val_loss : 0.6732, acc : 0.990, val_acc : 0.918
test_acc : 0.927
